<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/code/01RAD_Ex11__hands_on_empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle house data set



## Downloading the Kaggle house rent dataset

The dataset we will use comes from Kaggle:

- *House Rent Prediction Dataset*  
  https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset/data

To download directly from Kaggle inside this notebook you need a Kaggle
API token (see *Account ? API ? Create New Token* on Kaggle). The cell
below assumes you have configured your `KAGGLE_USERNAME` and
`KAGGLE_KEY` environment variables or placed `kaggle.json` in the
standard location.

Exampe of Auto NB - let's beat it

https://www.kaggle.com/code/sahityasetu/boosting-algorithms-for-machine-learning


In [1]:

# Download the Kaggle house rent dataset using kagglehub (no API key needed for public data)
try:
    import kagglehub  # lightweight helper for Kaggle datasets
except ImportError:  # pragma: no cover
    %pip install -q kagglehub
    import kagglehub

# Download latest version of the dataset; this returns a local directory path
path = kagglehub.dataset_download("iamsouravbanerjee/house-rent-prediction-dataset")
print("Path to dataset files:", path)


Path to dataset files: C:\Users\francji1\.cache\kagglehub\datasets\iamsouravbanerjee\house-rent-prediction-dataset\versions\9


In [2]:

from pathlib import Path
import pandas as pd

# `path` is a directory returned by kagglehub; locate the CSV inside it
dataset_dir = Path(path)
candidates = list(dataset_dir.rglob("House_Rent_Dataset.csv"))
if not candidates:
    raise FileNotFoundError(f"House_Rent_Dataset.csv not found under {dataset_dir}")

csv_path = candidates[0]
print("Loading data from:", csv_path)
house = pd.read_csv(csv_path)
print("Shape:", house.shape)
house.head()


Loading data from: C:\Users\francji1\.cache\kagglehub\datasets\iamsouravbanerjee\house-rent-prediction-dataset\versions\9\House_Rent_Dataset.csv
Shape: (4746, 12)


,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner



## Questions for a linear regression analysis of house rent

When building a linear regression model for rent, it is useful to think
in terms of a workflow:

1. **Understand the data**
   - What is the response variable (e.g. `Rent`)?  
     What are the main predictor types (numeric, categorical, locations,
     amenities)?
   - Are there obvious data quality issues (missing values, impossible
     values, outliers)?

2. **Preprocessing and feature engineering**
   - How should categorical variables (e.g. city, furnishing status,
     point of contact) be encoded for a linear model (one?hot encoding,
     target encoding, etc.)?
   - Which numeric variables might benefit from scaling (standardization
     or robust scaling), and why can this matter for regularized
     regression?
   - Are there interactions that are conceptually meaningful
     (e.g. `BHK , Size`, `City , DistanceFromMainArea`)?
   - Can we create more interpretable features (e.g. rent per square
     foot, distance to city centre bins)?

3. **Transformations of response and regressors**
   - Is the distribution of `Rent` highly skewed or heavy tailed? Would a
     log transformation (modeling $\log(\text{Rent})$) stabilize
     variance and make residuals closer to normal?
   - Do some predictors show non linear relationships with rent? Would
     polynomial terms, splines, or monotone transforms (log, square
     root) be appropriate?
   - Are there predictors that should be centered or standardized before
     creating interaction or polynomial terms?

4. **Model specification and selection**
   - Start with a simple baseline: which variables should be included in
     a first OLS model, and how do residual plots look?
   - How to compare alternative specifications
     (different sets of features, transformed vs untransformed variables)
     using cross validation or a validation set?
   - When is it useful to move from plain OLS to regularized models such
     as ridge or lasso (e.g. many correlated predictors, high variance)?

5. **Model evaluation and diagnostics**
   - How to check linear model assumptions: residual vs fitted plots,
     QQ plots, heteroscedasticity, influential observations?
   - Which error metrics are most relevant here
     (RMSE, MAE, MAPE)?  How do training and test errors compare
     (overfitting vs underfitting)?
   - Are there systematic groups of houses (by city, BHK, furnishing)
     for which the model performs much worse, suggesting missing
     structure or interactions?



### More detailed questions to explore

- **Preprocessing**
  - How should missing values be handled for each variable (impute,
    drop, or create explicit missing indicators)?
  - Do we need to cap or Winsorize extreme values of `Rent` or `Size`
    before fitting a linear model?
  - Are there rare categories (e.g. cities or furnishing statuses with
    very few observations) that should be grouped together?

- **Transformations and linearity**
  - Plot `Rent` (or $\log(\text{Rent})$) against key predictors:
    `Size`, `BHK`, `Bathroom`, `City`, etc.  Do the relationships look
    approximately linear after transformation?
  - Would modeling $\log(\text{Rent})$ make residuals more symmetric and
    reduce heteroscedasticity?

- **Multicollinearity and regularization**
  - Are some predictors strongly correlated (e.g. `Size` and `BHK`)?  How
    do VIFs and condition numbers look for the chosen design matrix?
  - How do ridge and lasso behave in this dataset in terms of coefficient
    shrinkage and variable selection?
  - Which predictors consistently get selected by lasso across
    cross?validation folds?

- **Model selection and validation**
  - How does test error change when we:
    1. Add more predictors,
    2. Add interaction terms,
    3. Add polynomial terms,
    4. Switch from OLS to ridge/lasso?
  - How to choose the final model: by minimum cross?validated RMSE,
    parsimony (fewest predictors), or domain interpretability?

Use these questions as a checklist to design your own modeling pipeline
for the house rent dataset using linear regression and its regularized
variants.
